In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer


In [2]:
df = pd.read_csv('movies.csv')

d:\recommendationSystem\venv\lib\site-packages\IPython\core\interactiveshell.py:3156: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [4]:
df = df[['original_title','overview']]
df.head(2)

,original_title,overview
0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,When siblings Judy and Peter discover an encha...


In [5]:
df.shape

(45466, 2)

let take few samples

In [6]:
# df = df.head(200) 
df.fillna('',inplace=True)

<h1>pre-process to remove numbers , punctuations, tags 

In [7]:
import re
def preprocess(text):
    text  = text.lower()
    t = re.findall('[a-z]+',text)
    t=' '.join(t)
    return t


In [8]:
df['overview']=df['overview'].apply(preprocess)

In [9]:
tf_idf = TfidfVectorizer(analyzer='word',stop_words='english',max_features=10000)

In [10]:
m=tf_idf.fit_transform(df['overview'].fillna(''))

In [11]:
m

<45466x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 1031207 stored elements in Compressed Sparse Row format>

In [12]:
d=pd.DataFrame(data = m.toarray(),columns = tf_idf.get_feature_names())

In [14]:
from sklearn.metrics.pairwise import cosine_similarity,linear_kernel

In [15]:
sim_mat = linear_kernel(d,d)

In [16]:
sim_mat

array([[1.        , 0.0165807 , 0.        , ..., 0.        , 0.00685624,
        0.        ],
       [0.0165807 , 1.        , 0.05520837, ..., 0.        , 0.02671951,
        0.01321592],
       [0.        , 0.05520837, 1.        , ..., 0.        , 0.01727485,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.00685624, 0.02671951, 0.01727485, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.01321592, 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [17]:
indices = pd.Series(df.index, index=df['original_title'])
indices

original_title
Toy Story                          0
Jumanji                            1
Grumpier Old Men                   2
Waiting to Exhale                  3
Father of the Bride Part II        4
                               ...  
رگ خواب                        45461
Siglo ng Pagluluwal            45462
Betrayal                       45463
Satana likuyushchiy            45464
Queerama                       45465
Length: 45466, dtype: int64

In [18]:
def getMovie(cos_sim,idx,k):
    score = list(enumerate(cos_sim[idx]))
    print('looking similar movies to ===',df.loc[idx]['original_title'])
    score = sorted(score , key=lambda x:x[1],reverse=True)
    movies_idx =  [i[0] for i in score[1:k]]
    # print(movies_idx)
    return df.iloc[movies_idx]['original_title']

In [20]:
getMovie(sim_mat,12481,8)

looking similar movies to === The Dark Knight


1328                                        Batman Returns
18252                                The Dark Knight Rises
15511                           Batman: Under the Red Hood
21194    Batman Unmasked: The Psychology of the Dark Kn...
40974    LEGO DC Comics Super Heroes: Batman: Be-Leaguered
150                                         Batman Forever
19792              Batman: The Dark Knight Returns, Part 1
Name: original_title, dtype: object